## DS 5110 Final Project - Hospital Mortality
### By: Elena Tsvetskova, Brian Chae, Ryan Viti (rrv7eb)

In [17]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

spark = SparkSession.builder \
        .master("local") \
        .appName("Team 3 Final Project") \
        .getOrCreate()

filename = "hospital_mortality.csv"

data = spark.read.csv(filename, inferSchema=True, header=True)
print("Number of rows in Mortality Dataset: {}".format(data.count()))

Number of rows in Mortality Dataset: 24503


In [18]:
data.printSchema()

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- hospital_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- elective_surgery: integer (nullable = true)
 |-- ethnicity: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height: double (nullable = true)
 |-- icu_admit_source: string (nullable = true)
 |-- icu_id: integer (nullable = true)
 |-- icu_stay_type: string (nullable = true)
 |-- icu_type: string (nullable = true)
 |-- pre_icu_los_days: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- apache_2_diagnosis: integer (nullable = true)
 |-- apache_3j_diagnosis: double (nullable = true)
 |-- apache_post_operative: integer (nullable = true)
 |-- arf_apache: integer (nullable = true)
 |-- gcs_eyes_apache: integer (nullable = true)
 |-- gcs_motor_apache: integer (nullable = true)
 |-- gcs_unable_apache: integer (nullable = true)
 |-- gcs_verbal_apache

## Data Cleaning and EDA

In [19]:
# empty columns and unique identifiers unnecessary for analysis

data = data.drop("_c83", "encounter_id", "patient_id", "hospital_id", "icu_id", "icu_stay_type", "pre_icu_los_days")

# Subsetting integer data for EDA purposes
integer_data = data.select([data.dtypes[i][0] for i in range(len(data.dtypes)) if (data.dtypes[i][1] == 'int') and (data.select(data[i]).distinct().count() >= 10)])

for i in integer_data.columns:
    integer_data.describe(i).show()
    integer_data.select(F.percentile_approx(i, [0.25, 0.75], 100000).alias("quantiles")).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             23308|
|   mean| 62.60936159258624|
| stddev|16.739045957199462|
|    min|                16|
|    max|                89|
+-------+------------------+

+---------+
|quantiles|
+---------+
| [53, 76]|
+---------+

+-------+------------------+
|summary|apache_2_diagnosis|
+-------+------------------+
|  count|             24025|
|   mean|187.11779396462018|
| stddev| 85.93823690825164|
|    min|               101|
|    max|               308|
+-------+------------------+

+----------+
| quantiles|
+----------+
|[113, 301]|
+----------+

+-------+-----------------+
|summary|heart_rate_apache|
+-------+-----------------+
|  count|            24274|
|   mean|98.78104144351981|
| stddev|31.05756946762657|
|    min|               30|
|    max|              178|
+-------+-----------------+

+---------+
|quantiles|
+---------+
|[82, 119]|
+---------+

+-------+-----------------+
|summar

In [24]:
len(integer_data.columns)

42

In [28]:
non_integer_data = data.select([i for i in data.columns if i not in integer_data.columns])
non_integer_data.columns

['bmi',
 'elective_surgery',
 'ethnicity',
 'gender',
 'height',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type',
 'pre_icu_los_days',
 'weight',
 'apache_3j_diagnosis',
 'apache_post_operative',
 'arf_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'intubated_apache',
 'temp_apache',
 'ventilated_apache',
 'd1_sysbp_noninvasive_min',
 'd1_temp_max',
 'd1_temp_min',
 'd1_potassium_max',
 'd1_potassium_min',
 'apache_4a_hospital_death_prob',
 'apache_4a_icu_death_prob',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis',
 'apache_3j_bodysystem',
 'apache_2_bodysystem',
 'hospital_death']

In [15]:
data = data.na.drop(subset="hospital_death")
data.select("hospital_death").groupBy("hospital_death").count().show()

+--------------+-----+
|hospital_death|count|
+--------------+-----+
|          null|    1|
|             1| 1951|
|             0|22551|
+--------------+-----+

+--------------+-----+
|hospital_death|count|
+--------------+-----+
|             1| 1951|
|             0|22551|
+--------------+-----+

